# 🧾 Python Fundamentals — 09 Typing, TypedDict & Pydantic v2

Static typing improves **clarity, validation, and IDE support**.
Python’s type hints are optional but highly recommended for modern codebases.

This notebook introduces:
- Type hints and annotations
- `TypedDict`, `Literal`, `Optional`, `Annotated`
- Pydantic v2 for runtime validation
- Differences between `TypedDict` (static typing) and Pydantic (runtime validation)

## 1️⃣ Type hints — annotating variables and functions

In [1]:
from typing import List, Dict, Optional

def greet(name: str, age: Optional[int] = None) -> str:
    if age is not None:
        return f"Hello {name}, you are {age} years old."
    return f"Hello {name}."

message = greet("Alice", 30)
print(message)

Hello Alice, you are 30 years old.


- Type hints don’t affect runtime — they’re for tooling, IDEs, and static analysis (e.g., `mypy`, `pyright`).

## 2️⃣ `TypedDict` — type-safe dictionaries

In [2]:
from typing import TypedDict

class UserInfo(TypedDict):
    name: str
    age: int
    email: str

user: UserInfo = {"name": "Alice", "age": 30, "email": "alice@example.com"}
print(user)

{'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}


`TypedDict` describes the **shape of a dictionary** for static type checkers — it does not validate at runtime.

### Optional keys

In [3]:
class PartialUser(TypedDict, total=False):
    name: str
    age: int

partial: PartialUser = {"name": "Bob"}
print(partial)

{'name': 'Bob'}


## 3️⃣ Literals and Annotated types

In [4]:
from typing import Literal, Annotated

Status = Literal["pending", "approved", "rejected"]

def process(status: Status) -> None:
    print("Processing status:", status)

process("approved")

Processing status: approved


`Literal` restricts allowed values to a fixed set — useful for enums or state machines.
`Annotated` can attach metadata for validators or frameworks (like Pydantic).

## 4️⃣ Pydantic v2 — runtime validation and parsing

In [5]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str
    price: float = Field(..., gt=0)
    in_stock: bool = True

item = Product(name="Laptop", price=1299.99)
print(item)

# Access as dict
print(item.model_dump())

name='Laptop' price=1299.99 in_stock=True
{'name': 'Laptop', 'price': 1299.99, 'in_stock': True}


If invalid data is passed, Pydantic raises a `ValidationError` with detailed messages.

In [6]:
try:
    Product(name="BadItem", price=-10)
except Exception as e:
    print(e)

1 validation error for Product
price
  Input should be greater than 0 [type=greater_than, input_value=-10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## 5️⃣ TypedDict vs Pydantic — key differences

| Feature | TypedDict | Pydantic |
|----------|------------|-----------|
| Purpose | Static type checking | Runtime validation + parsing |
| Runtime cost | None | Small (validation) |
| IDE awareness | ✅ | ✅ |
| Data mutation | Mutable | Mutable/immutable configurable |
| Use case | Internal state typing | API input/output, configs |

- Use `TypedDict` for lightweight internal contracts.
- Use **Pydantic** for external I/O: configs, requests, API schemas.

### 🧭 Summary
- Type hints clarify intent and improve tooling
- `TypedDict` describes dict structure (static only)
- `Pydantic` validates and parses at runtime
- Use both together: TypedDict for **internal state**, Pydantic for **external data boundaries**